In [ ]:
# 출처 : https://wikidocs.net/233344
# LangChain 설치 및 업데이트
#!pip install -U langchain langchain-community langchain-experimental langchain-core langchain-openai langsmith langchainhub python-dotenv unstructured chromadb faiss-cpu rank_bm25 python-docx sqlalchemy

In [ ]:
# 루트경로에 .env 파일을 만들고, OPENAI_API_KEY='{API_KEY}' 식으로 입력한다.
# API 키를 환경변수로 관리하기 위한 .env설정 파일 로딩
import os
from dotenv import load_dotenv

load_dotenv() # API 키 정보 로드
print(f"[API KEY]\n{os.environ['OPENAI_API_KEY']}")

In [2]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

# ChatOpenAI 모델을 인스턴스화합니다.
model = ChatOpenAI()

# 주어진 토픽에 대한 농담을 요청하는 프롬프트 템플릿을 생성합니다.
prompt = ChatPromptTemplate.from_template("{topic} 에 대하여 3문장으로 설명해줘.")

# 프롬프트와 모델을 연결하여 대화 체인을 생성합니다.
chain = prompt | model

In [3]:
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [5]:
chain.output_schema.schema()

{'title': 'ChatOpenAIOutput',
 'anyOf': [{'$ref': '#/definitions/AIMessage'},
  {'$ref': '#/definitions/HumanMessage'},
  {'$ref': '#/definitions/ChatMessage'},
  {'$ref': '#/definitions/SystemMessage'},
  {'$ref': '#/definitions/FunctionMessage'},
  {'$ref': '#/definitions/ToolMessage'}],
 'definitions': {'ToolCall': {'title': 'ToolCall',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'args': {'title': 'Args', 'type': 'object'},
    'id': {'title': 'Id', 'type': 'string'}},
   'required': ['name', 'args', 'id']},
  'InvalidToolCall': {'title': 'InvalidToolCall',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'args': {'title': 'Args', 'type': 'string'},
    'id': {'title': 'Id', 'type': 'string'},
    'error': {'title': 'Error', 'type': 'string'}},
   'required': ['name', 'args', 'id', 'error']},
  'AIMessage': {'title': 'AIMessage',
   'description': 'Message from an AI.',
   'type': 'object',
   'proper

In [6]:
# stream: 실시간 출력
# 이 함수는 chain.stream 메서드를 사용하여 주어진 토픽에 대한 데이터 스트림을 생성하고, 
# 이 스트림을 반복하여 각 데이터의 내용(content)을 즉시 출력합니다. 
# end="" 인자는 출력 후 줄바꿈을 하지 않도록 설정하며, flush=True 인자는 출력 버퍼를 즉시 비우도록 합니다. 
# 이는 스트리밍 데이터를 실시간으로 처리할 때 유용하게 사용됩니다.
# chain.stream 메서드를 사용하여 '멀티모달' 토픽에 대한 스트림을 생성하고 반복합니다.
for s in chain.stream({"topic": "멀티모달"}):
    # 스트림에서 받은 데이터의 내용을 출력합니다. 줄바꿈 없이 이어서 출력하고, 버퍼를 즉시 비웁니다.
    print(s.content, end="", flush=True)

멀티모달은 여러 가지 다른 형태의 수단을 사용하여 정보를 전달하거나 사용자와 상호작용하는 방식을 의미합니다. 예를 들어 음성, 텍스트, 이미지, 동영상 등 다양한 매체를 활용하여 사용자에게 정보를 제공하거나 응답하는 것이 가능합니다. 멀티모달은 사용자 경험을 향상시키고 보다 효율적으로 정보를 전달할 수 있도록 도와줍니다.

In [7]:
# invoke: 호출
# chain 객체의 invoke 메서드는 주제를 인자로 받아 해당 주제에 대한 처리를 수행합니다. 
# 이 예제에서는 topic 키에 'ChatGPT' 값을 가진 딕셔너리를 invoke 메서드에 전달하고 있습니다.

# chain 객체의 invoke 메서드를 호출하고, 'ChatGPT'라는 주제로 딕셔너리를 전달합니다.
chain.invoke({"topic": "ChatGPT"})

AIMessage(content='ChatGPT는 OpenAI에서 개발된 대화형 인공지능 모델로, 사용자와 대화를 나누며 다양한 주제에 대해 대화할 수 있습니다. 이 모델은 텍스트 입력을 받아들이고 자연어 이해 기술을 활용하여 응답을 생성하며, 사용자들의 질문에 대답하거나 대화를 이어나갈 수 있습니다. ChatGPT는 기계 학습과 인공지능 기술의 발전을 보여주며, 실제 상황에서 대화를 주도하기 위한 다양한 응용 분야에서 사용될 수 있습니다.', response_metadata={'token_usage': {'completion_tokens': 180, 'prompt_tokens': 24, 'total_tokens': 204}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ca303e2b-3227-4c47-844e-d72dfd9e65fe-0')

In [9]:
# async stream 중간 단계 디버깅

from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
# prompt 설정
prompt = ChatPromptTemplate.from_template(template)

# retriever 설정
vectorstore = FAISS.from_texts(
    ["테디는 나이가 48살입니다. 현재 하는 일을 컴퓨터프로그래머입니다. 살고 있는 곳은 대한민국 제주도 제주시 입니다."], 
    embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

# 모델 설정
model = ChatOpenAI()

# chain 구성
retrieval_chain = (
    {
        "context": retriever.with_config(run_name="Docs"),
        "question": RunnablePassthrough(),
    }
    | prompt
    | model
    | StrOutputParser()
)

# 비동기로 중간단계 디버깅함.
async for chunk in retrieval_chain.astream_log(
    "테디가 살고 있는 곳은 어딘가요?", include_names=["Docs"]
):
    print("-" * 40)
    print(chunk)

----------------------------------------
RunLogPatch({'op': 'replace',
  'path': '',
  'value': {'final_output': None,
            'id': 'a2aa0a0d-8ce9-4b46-a84e-e6c02976a747',
            'logs': {},
            'name': 'RunnableSequence',
            'streamed_output': [],
            'type': 'chain'}})
----------------------------------------
RunLogPatch({'op': 'add',
  'path': '/logs/Docs',
  'value': {'end_time': None,
            'final_output': None,
            'id': 'af71ee71-f3a5-4631-9b83-106265090d70',
            'metadata': {},
            'name': 'Docs',
            'start_time': '2024-05-08T02:34:02.318+00:00',
            'streamed_output': [],
            'streamed_output_str': [],
            'tags': ['map:key:context', 'FAISS', 'OpenAIEmbeddings'],
            'type': 'retriever'}})
----------------------------------------
RunLogPatch({'op': 'add',
  'path': '/logs/Docs/final_output',
  'value': {'documents': [Document(page_content='테디는 나이가 48살입니다. 현재 하는 일을 컴퓨터프로그래머

In [10]:
# Parallel: 병렬성
# langchain_core.runnables 모듈의 RunnableParallel 클래스를 사용하여 두 가지 작업을 병렬로 실행

from langchain_core.runnables import RunnableParallel

# 모델 설정
model = ChatOpenAI()

# {country} 의 수도를 물어보는 체인을 생성합니다.
chain1 = ChatPromptTemplate.from_template("{country} 의 수도는 어디야?") | model

# {country} 의 면적을 물어보는 체인을 생성합니다.
chain2 = ChatPromptTemplate.from_template("{country} 의 면적은 얼마야?") | model
# 위의 2개 체인을 동시에 생성하는 병렬 실행 체인을 생성합니다.
combined = RunnableParallel(capital=chain1, area=chain2)

# 주어진 'country'에 대해 'combined' 객체의 'invoke' 메서드를 호출합니다.
response = combined.invoke({"country": "대한민국"})
print(response)

{'capital': AIMessage(content='대한민국의 수도는 서울입니다.', response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 23, 'total_tokens': 38}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d2b13b3c-9266-4036-9c58-7064feab70cf-0'), 'area': AIMessage(content='대한민국의 면적은 약 100,363 제곱킬로미터 입니다.', response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 25, 'total_tokens': 53}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-afed9a7e-f2ca-4f15-814b-bcff6dbef3a9-0')}


In [13]:
# RunnablePassthrough 적용
# RunnablePassthrough는 입력을 변경하지 않거나 추가 키를 더하여 전달할 수 있습니다.
from langchain.prompts import PromptTemplate

chain1 = (
    {"num": RunnablePassthrough()}  # 입력변경하지 않고 pass
    | PromptTemplate.from_template("{num} 의 10배는?\n답변(결과만): ")
    | ChatOpenAI()
)
chain2 = (
    {"num": RunnablePassthrough.assign(mult=lambda x: x["num"] * 3)}  # mult라는 출력값 = num입력에 *3 값도 출력.
    | PromptTemplate.from_template("{num} 의 1/10배는?\n답변(결과만): ")
    | ChatOpenAI()
)

combined_chain = RunnableParallel(a=chain1, b=chain2)
response = combined_chain.invoke({"num": 10})

print(response)


{'a': AIMessage(content='100', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 31, 'total_tokens': 32}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3494c6da-8bb9-4d0b-80b8-92a1a2153f54-0'), 'b': AIMessage(content="{'num': 1, 'mult': 3}", response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 39, 'total_tokens': 51}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0c401d37-9640-4d22-bf5d-706dd366eab4-0')}
